In [1]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
import torch
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
def load_images_from_folder(folder):
    images = []
    for file in os.listdir(folder):
        img = cv.imread(os.path.join(folder,file))
        if(img is not None):
            images.append(img)
    return images

In [4]:
def resize_and_flatten(img, target_size=(64,64)):
    resized_img = cv.resize(img, target_size)
    flattened_img = resized_img.reshape(target_size[0] * target_size[1] * 3)
    return flattened_img

In [5]:
cat_data = []
for img in load_images_from_folder("datasets/cat_v_dog/Cat"):
    cat_data.append(resize_and_flatten(img))
dog_data = []
for img in load_images_from_folder("datasets/cat_v_dog/Dog"):
    dog_data.append(resize_and_flatten(img))

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\git\x08achelor-thesis\\datasets\\cat_v_dog\\Cat'

In [ ]:
print(len(cat_data) + " cat images")
print(len(dog_data) + " dog images")

In [ ]:
X = []
y = []

for entity in cat_data:
    X.append(entity)
    y.append(1)

for entity in dog_data:
    X.append(entity)
    y.append(0)

np.array(X).shape, np.array(y).shape

In [ ]:
class MyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64*64*3, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 2),
            nn.Sigmoid()
        )
    def forward(self, x, **kwargs):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
net = NeuralNetClassifier(
    MyNN,
    max_epochs=10,
    lr=0.1,
    iterator_train__shuffle=True,
)


In [ ]:
params = {    
    'lr': [0.001, 0.01, 0.0001],
    'max_epochs': [1,2,5, 10, 20, 25],
}
gs = GridSearchCV(net, params, cv=2)
gs.fit(np.array(X, dtype='float32'), np.array(y))
pd.DataFrame(gs.cv_results_)